# Inception_V1 implementation using CIFAR10 dataset

In [19]:
import numpy as np
import keras
from keras import layers
import keras.backend as K
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.callbacks import CSVLogger, EarlyStopping
from keras.initializers import glorot_uniform
import cv2

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [20]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = np.array([cv2.resize(img, (64,64)) for img in X_train[:10000,:,:,:]])
X_test = np.array([cv2.resize(img, (64,64)) for img in X_test[:1000,:,:,:]])

n_classes = 10

Y_train = np_utils.to_categorical(Y_train[:10000,:], n_classes)
Y_test = np_utils.to_categorical(Y_test[:1000,:], n_classes)

mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

In [1]:
def inception_block(input_x, filters):
    f1, f2, f3, f4, f5, f6 = filters
    
    X = input_x
    
    conv1_1x1 = layers.Conv2D(f1, kernel_size=(1,1), padding='same', kernel_initializer=glorot_uniform())(input_x)
    conv1_1x1 = layers.Activation('relu')(conv1_1x1)
    
    conv2_1x1 = layers.Conv2D(f2, kernel_size=(1,1), padding='same', kernel_initializer=glorot_uniform())(input_x)
    conv2_1x1 = layers.Activation('relu')(conv2_1x1)
    conv2_3x3 = layers.Conv2D(f3, kernel_size=(3,3), padding='same', kernel_initializer=glorot_uniform())(conv2_1x1)
    conv2_3x3 = layers.Activation('relu')(conv2_3x3)
    
    conv3_1x1 = layers.Conv2D(f4, kernel_size=(1,1), padding='same', kernel_initializer=glorot_uniform())(input_x)
    conv3_1x1 = layers.Activation('relu')(conv3_1x1)
    conv3_5x5 = layers.Conv2D(f5, kernel_size=(5,5), padding='same', kernel_initializer=glorot_uniform())(conv3_1x1)
    conv3_5x5 = layers.Activation('relu')(conv3_5x5)
    
    pool = layers.MaxPool2D(pool_size=(3,3), strides=(1,1), padding="same")(input_x)
    conv_pool_1x1 = layers.Conv2D(f6, kernel_size=(1,1), padding='same', kernel_initializer=glorot_uniform())(pool)
    conv_pool_1x1 = layers.Activation('relu')(conv_pool_1x1)
    
    output = layers.Concatenate(axis=-1)([conv1_1x1, conv2_3x3, conv3_5x5, conv_pool_1x1])
    
    return output
    

In [22]:
def inception_v1(input_shape, n_classes):
    
    input_x = layers.Input(input_shape)
    
    X = input_x
    X = layers.ZeroPadding2D((3,3))(input_x)
    
    X = layers.Conv2D(64, kernel_size=(7,7), padding="valid", strides=(2,2), kernel_initializer=glorot_uniform())(X)
    X = layers.Activation('relu')(X)
    
    X = layers.ZeroPadding2D((1,1))(X)
    
    X = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid")(X)
    
    #Local Response normalization skipped
    
    X = layers.Conv2D(64, kernel_size=(1,1), padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(192, kernel_size=(3,3), padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(X)
    X = layers.Activation('relu')(X)
    
    #Local Response normalization skipped
    
    X = layers.ZeroPadding2D((1,1))(X)
    
    X = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid")(X)
    
    X = inception_block(X, [64, 96, 128, 16, 32, 32])
    
    X = inception_block(X, [128, 128, 192, 32, 96, 64])
    
    X = layers.ZeroPadding2D((1,1))(X)
    
    X = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid")(X)
    
    X = inception_block(X, [192, 96, 208, 16, 48, 64])
    
    #pool_size = (5,5) , strides=(3,3) in orignal implementation
    aux_output1 = layers.AveragePooling2D(pool_size=(3,3), strides=(1,1), padding="valid")(X)
    aux_output1 = layers.Conv2D(128, kernel_size=(1,1), strides=(1,1), padding="same")(aux_output1)
    
    aux_output1 = layers.Flatten()(aux_output1)
    
    aux_output1 = layers.Dense(1024, activation='relu')(aux_output1)
    aux_output1 = layers.Dropout(0.7)(aux_output1)
    
    aux_output1 = layers.Dense(n_classes, activation='softmax')(aux_output1)
    
    X = inception_block(X, [160, 112, 224, 24, 64, 64])
    
    X = inception_block(X, [128, 128, 256, 24, 65, 64])
    
    X = inception_block(X, [112, 144, 288, 32, 64, 64])
    
    #pool_size = (5,5) , strides=(3,3) in orignal implementation
    aux_output2 = layers.AveragePooling2D(pool_size=(3,3), strides=(1,1), padding="valid")(X)
    aux_output2 = layers.Conv2D(128, kernel_size=(1,1), strides=(1,1), padding="same")(aux_output2)
    
    aux_output2 = layers.Flatten()(aux_output2)
    
    aux_output2 = layers.Dense(1024, activation='relu')(aux_output2)
    aux_output2 = layers.Dropout(0.7)(aux_output2)
    
    aux_output2 = layers.Dense(n_classes, activation='softmax')(aux_output2)
    
    X = inception_block(X, [256, 160, 320, 32, 128, 128])
    
    X = layers.ZeroPadding2D((1,1))(X)
    
    X = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid")(X)
    
    X = inception_block(X, [256, 160, 320, 32, 128, 128])
    
    X = inception_block(X, [384, 192, 384, 48, 128, 128])
    
    X = layers.AveragePooling2D(pool_size=(2,2), strides=(1,1), padding="valid")(X)
    
    X = layers.Flatten()(X)
    
    X = layers.Dropout(0.4)(X)
    
    X = layers.Dense(n_classes, activation='softmax')(X)
    
    model = keras.Model(inputs=input_x, outputs=X)
    
    return model
    
    
    

In [23]:
inception_v1_model = inception_v1(input_shape = (64, 64, 3), n_classes=10)

In [24]:
inception_v1_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
inception_v1_model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Epoch 1/10
10000/10000 [==============================] - 137s 14ms/step - loss: 1.5180 - acc: 0.4137
Epoch 2/10
10000/10000 [==============================] - 136s 14ms/step - loss: 1.4184 - acc: 0.4557
Epoch 3/10
10000/10000 [==============================] - 136s 14ms/step - loss: 1.3236 - acc: 0.5076
Epoch 4/10
10000/10000 [==============================] - 136s 14ms/step - loss: 1.2134 - acc: 0.5545
Epoch 5/10
10000/10000 [==============================] - 136s 14ms/step - loss: 1.1088 - acc: 0.5939
Epoch 6/10
10000/10000 [==============================] - 136s 14ms/step - loss: 1.0411 - acc: 0.6250
Epoch 7/10
10000/10000 [==============================] - 136s 14ms/step - loss: 0.9564 - acc: 0.6546
Epoch 8/10
10000/10000 [==============================] - 136s 14ms/step - loss: 0.8650 - acc: 0.6877
Epoch 9/10
10000/10000 [==============================] - 136s 14ms/step - loss: 0.7785 - acc: 0.7185
Epoch 10/10
10000/10000 [==============================] - 136s 14ms/step - loss: 

In [17]:
score, accuracy = inception_v1_model.evaluate(X_test, Y_test)

1000/1000 [==============================] - 4s 4ms/step


In [18]:
print accuracy

0.611
